In [2]:
!pip install torch torchvision torchaudio transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset


In [46]:
# Load dataset
dataset = load_dataset("quora")
# Reduce dataset size after loading (random 10k samples)
dataset = dataset["train"].shuffle(seed=42).select(range(5000))


# Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

In [47]:
def preprocess_function(examples):
    q1_list = [q["text"][0] for q in examples["questions"]]  # Extract first question
    q2_list = [q["text"][1] for q in examples["questions"]]  # Extract second question

    inputs = ["paraphrase: " + q1 for q1 in q1_list]
    targets = q2_list

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



In [48]:
print(dataset)

Dataset({
    features: ['questions', 'is_duplicate'],
    num_rows: 5000
})


In [49]:
# Process dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Load model
model = T5ForConditionalGeneration.from_pretrained("t5-small")


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [50]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",  # Disable evaluation
    save_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=500,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    report_to="none",
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [51]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [52]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)
trainer.train()

Step,Training Loss
500,0.414400
1000,0.315500
1500,0.290200


TrainOutput(global_step=1875, training_loss=0.32777216796875, metrics={'train_runtime': 16633.3709, 'train_samples_per_second': 0.902, 'train_steps_per_second': 0.113, 'total_flos': 507531755520000.0, 'train_loss': 0.32777216796875, 'epoch': 3.0})

In [40]:
!pip install nltk

In [41]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [45]:
import nltk
nltk.download('punkt_tab') # Download the 'punkt_tab' resource


from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score

# Function to generate predictions and calculate both accuracy and BLEU score
def evaluate_model(model, tokenizer, dataset, max_length=128):
    predictions = []
    references = []

    for example in dataset:
        input_text = "paraphrase: " + example["questions"]["text"][0]
        input_ids = tokenizer(input_text, return_tensors="pt", max_length=max_length, truncation=True).input_ids.to(model.device)

        # Generate prediction
        output_ids = model.generate(input_ids, max_length=max_length)
        prediction = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # Get reference
        reference = example["questions"]["text"][1]

        # Append to lists
        predictions.append(prediction)
        references.append(reference)

    # Calculate accuracy
    accuracy = accuracy_score(references, predictions)

    # Calculate BLEU score
    tokenized_predictions = [word_tokenize(pred) for pred in predictions]
    tokenized_references = [[word_tokenize(ref)] for ref in references]
    bleu_scores = [sentence_bleu(ref, pred) for ref, pred in zip(tokenized_references, tokenized_predictions)]
    average_bleu = sum(bleu_scores) / len(bleu_scores)

    return accuracy, average_bleu

# Evaluate model
accuracy, average_bleu = evaluate_model(model, tokenizer, dataset)
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Average BLEU score: {average_bleu:.4f}")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-g

Accuracy: 1.05%
Average BLEU score: 0.1276
